# Demonstration of using React within FastHTML

>This example does *not* require a build step, as we are not using JSX for the templating.

Jupyter instantiation code borrowed from Isaac (thanks!)

In [1]:
from IPython.display import display, HTML
from fasthtml.common import *
from fasthtml.jupyter import *

def create_server(app,rt):
    if IN_NOTEBOOK:
        for port in range(8000,8030):   
            if 'server' in globals():
                print(f"Server already running on port {globals()['server'].port} - stopping it")
                globals()['server'].stop()
            if is_port_free(port):
                server = JupyUvi(app, port=port)

                def HShow(comp, app):
                    @app.get('/')
                    def get(): return comp
                    display(HTML(f'<a href="http://localhost:{port}/" target="_blank">Open in new tab</a>'))
                    return HTMX("/",port=port)
                print(f"Starter server on port {port}")
                Show = partial(HShow, app=app)
                return app, rt, server, HShow, Show

In [2]:
react_cdn = Script(src="https://unpkg.com/react@18/umd/react.development.js", crossorigin=True, type="text/javascript")
react_dom_cdn = Script(src="https://unpkg.com/react-dom@18/umd/react-dom.development.js", crossorigin=True, type="text/javascript")

app, rt = fast_app(hdrs=[react_cdn, react_dom_cdn])
app, rt, server, HShow, Show = create_server(app, rt)

Starter server on port 8001


Define a React method, creates a standard button component that increments on click. Returns a wrapper Div with a unique ID that will act as the root node for this react island instance.

Using the useEffect() hook to get the approximate time taken to mount & hydrate the component from clicking the swap button.

In [3]:
def React():
    id=unqid()
    script = Script("""
const Count = () => {
    const [count, setCount] = React.useState(0);

    React.useEffect(() => {
        console.timeEnd('mount')
    }, []);

    return (
        React.createElement('button', {onClick: () => setCount(count+1), 'hx-get': '/test', 'hx-trigger': 'load', 'hx-target': 'main', 'hx-swap': 'beforeend'}, `Count: ${count}`)
    )
}

const root = ReactDOM.createRoot(document.getElementById('%s'));
root.render(React.createElement(Count, null, null));                   
""" % id)
    return Div(script, id=id)

@rt
def swap():
    return React()

@rt
def test():
    return Blockquote("This is rendered via a swapped-in react component!")

Show(Container(Button("Swap Me", hx_get="/swap", hx_swap="outerHTML", onclick="console.time('mount')")))

Time to mount from HTMX swap averaging between 10-18ms, with no flickering as the initial render is handled through javascript.

By putting the initial rendering script inside the div acting as the root node for React, the script itself is replaced by the component code upon mounting.

I like this as it acts as a clean approach, reducing the amount of redundant scripts in the tree if they only need to be called once. Might be a downside to this though, so will investigate further.

# Custom component for rendering React components in FastHTML

*WIP*

## Steps

Define the input -> output structure for the method

Takes an FT component:

```python
Function(*c, **kwargs)
```

And returns a structured string to be rendered via React.createElement:

```js
React.createElement('tagname', {attr: 'value'}, child1, child2, child3)
```

>**Notes**
> * Attribute names MUST be camelCase
> * Value is standard html attribute value syntax, except function calls which must NOT be a string
> * Children MUST be static, alternative rendering can be in the form of a list of children elements or a string/ JS template string to render variables

## Function Outlining


In [4]:
# Method Definition

# Default the tagname to 'div' if omitted
# Default children to None, optional tuple of FT components
# Standard FT **kwargs
# Optional state dict, consisting of name, value pairs

from fastcore.meta import delegates
import re

@delegates(ft_hx, keep=True)
def ft_react(tag:str='div', *c, state:dict=None, **kwargs):

    # Create unique ID for self mounting
    id = kwargs.pop("id", unqid())

    # Compose list of state strings
    init_state = []

    for s in state:
        init_state.append(f"const [{s}, set{s.title()}] = React.useState({state.get(s, None)});")

    # Format children - if string, keep as is for innerHTML
    # If not a string, assume its an FT instance

    #TODO: format & test FT children rendering
    # if c:
    #     for child in list(c):
    #         child = repr(child)

    fmt_children = ", ".join((repr(s) if "`" not in s else s for s in list(c))) 
    
    # Format kwargs
    new_kwargs = {}
    if kwargs:
        if 'cls' in kwargs: 
            new_kwargs['className'] = repr(kwargs.pop('cls'))

        for k,v in kwargs.items():
            k = k.replace('_', '-')
            # If Kwarg starts with `on`, assume its a function & format to React/JS syntax
            if k.startswith('on'):
                key_templ = 'on' + k[2:3].upper() + k[3:]
                func_templ = "() => {%s}"

                if isinstance(v, list):
                    inner_templ = "\n".join(f"set{list(state)[idx].title()}({val});" for idx, val in enumerate(v))
                else:
                    inner_templ = f"set{re.split(r'\W', v)[0].title()}({v});"

                new_kwargs[key_templ] = func_templ % inner_templ
            else:
                new_kwargs[repr(k)] = repr(v)

    fmt_kwargs = ", ".join(f"{k}" + ": " + f"{v}" for k,v in new_kwargs.items())

    # Temporary timing string to get time to mount+hydrate after swap
    # tmp_timing = "React.useEffect(() => {console.timeEnd('custom')}, []);"
                
    # Compose setup function for React component
    func_str = f"const {tag.title()} = () => {{{'\n'.join(init_state)}\
    return React.createElement('{tag}', {{ {fmt_kwargs} }}, { fmt_children })}};"

    # Create root & render string
    render_str = f"const root = ReactDOM.createRoot(document.getElementById('{id}'));\
    root.render(React.createElement({tag.title()}, null, null));"

    # Define Script with all pieces in order
    setup_script = Script(func_str+render_str)

    # Return container node, as a `div` with unique id matching the root element to mount to
    # Pass in the setup_script so that, upon render, the script executes and is removed
    
    # Script executes -> mounts to root div -> script is replaced with new element
    return Div(setup_script, id=id, onload="htmx.process(this)")

> Setting the onload inline function to `htmx.process(this)` ensures the react component is properly instantiated by HTMX whenever it is inserted into the DOM

In [5]:
Show(ft_react('button', "`Count: ${count}`", state={"count": 0}, onclick="count+5", cls="secondary outline"))

## Swapping Experiments

In [6]:

@rt
def custom_swap():
    return ft_react("button", "`Count: ${count}`", state={"count": 0}, onclick="count+5", cls="secondary outline")

Show(Container(H3("HTMX swapping test"), Div(Button("Swap me", hx_get="/custom_swap", hx_swap="outerHTML"))))

>Timing hovering between 14ms - 26ms with no flickering, works as intended

## Testing HX Attributes within custom React method

In [7]:
Show(ft_react('button', "`Value: ${count}`", state={"count": 1}, onclick="count+count", cls="outline", hx_trigger="load", hx_get="/test", hx_swap="beforeend", hx_target="body"))

## Multiple Values w/ Reactive Bindings

In [8]:
Show(ft_react('button', "`Values: x=${x}, y=${y}, z=${x+y}`", state={"x": 1, "y": 1, "z": "x+y"}, onclick=["x+1", "y+1"]))

## Exerimenting with a raw method

Building and expanding upon the FT component will be a priority, but having a set component that takes a react script and properly renders a processed and isolated react component from any react code will be very useful too.

This will allow integration of third party react libraries in an intuative, easy to implement FT method.

It should have options tied to it, for example:

- Allow for choosing between JSX and React.createElement functions (will need to toggle a babel cdn to properly process the JSX within the script for this)

## Testing raw function Structure

### Demo React component w/o JSX:

In [20]:
demo_comp = """
        const Clock = () => {
        const [time, setTime] = React.useState(Date.now())

        React.useEffect(() => {
        const interval = setInterval(() => {
            setTime(Date.now());
        }, 1000);

        return () => clearInterval(interval);
    }, [time]);

        return React.createElement('span', {}, `${new Date(time)}`);
    }
"""

In [21]:
def ft_react_raw(code:str='', jsx:bool=False, comp_name:str='default', id:str=None):

    if not id: id = unqid()
    
    babel_cdn = Script(src="https://unpkg.com/@babel/standalone/babel.min.js")

    render_scr = """const root = ReactDOM.createRoot(document.getElementById('%s'));""" % id

    if jsx:
        render_scr += "root.render(<%s />);" % comp_name.title()
    else:
        render_scr += "root.render(React.createElement(%s, null, null));" % comp_name.title()
    
    main_scr = Script(code + render_scr, type="text/babel" if jsx else 'module', data_type="module" if jsx else '')

    if jsx: 
        return (babel_cdn, Div(main_scr, id=id)) 
    else:
        return Div(main_scr, id=id)

In [22]:
Show(ft_react_raw(demo_comp, comp_name="clock"))

### Demo React Component with JSX

In [17]:
jsx_comp = """
        const Clock = () => {
        const [time, setTime] = React.useState(Date.now())

        React.useEffect(() => {
        const interval = setInterval(() => {
            setTime(Date.now());
        }, 1000);

        return () => clearInterval(interval);
    }, [time]);

        return (<span>{new Date(time).toString()}</span>)
    }
"""

In [18]:
Show(ft_react_raw(jsx_comp, comp_name="clock", jsx=True))